### GETTING DATA

In [1]:
import os
import os.path 
import pandas as pd
import numpy as np
import re

In [2]:
# creating a list containing all folder names of the dataset

folders = ["business", "entertainment", "food", "graphics", "historical", 
           "medical", "politics", "space", "sport", "technologie"]

In [3]:
# Changing the current working dirctory to the dataset directory

os.chdir("C:/Users/fetih.kaya/Desktop/dataset") 

In [4]:
# Verifying whether the current directory changed to the desier directory or not

print(os.getcwd())

C:\Users\fetih.kaya\Desktop\dataset


In [5]:
# generating  a list call paths containg all the detailed paths for accessing the folders of the dataset 

paths = []
for i in folders:
    paths.append(os.getcwd()+'/'+i)

In [6]:
# printing the path just for verification

print(paths)

['C:\\Users\\fetih.kaya\\Desktop\\dataset/business', 'C:\\Users\\fetih.kaya\\Desktop\\dataset/entertainment', 'C:\\Users\\fetih.kaya\\Desktop\\dataset/food', 'C:\\Users\\fetih.kaya\\Desktop\\dataset/graphics', 'C:\\Users\\fetih.kaya\\Desktop\\dataset/historical', 'C:\\Users\\fetih.kaya\\Desktop\\dataset/medical', 'C:\\Users\\fetih.kaya\\Desktop\\dataset/politics', 'C:\\Users\\fetih.kaya\\Desktop\\dataset/space', 'C:\\Users\\fetih.kaya\\Desktop\\dataset/sport', 'C:\\Users\\fetih.kaya\\Desktop\\dataset/technologie']


In [7]:
# creating two list texts and labels to store the detailed content and
# and the category of the news

texts = []
labels = []

In [8]:
#getting the path of a folders one by one from the list called paths[]
for path in paths:
    
#getting filenames one by one from the list of all files under a directory called path
    for filename in os.listdir(path):
        
#opening the file in read only mode with encoding latin1
        with open(path+"/"+filename,"r", encoding = "latin") as file:
        
#after reading the file contents are stored in a variable called data
            data = file.read()
    
#after removing all newline characters and carriage return from the  data,it is being added at the end of the list texts
            data = data.replace("\n"," ").replace('\r',' ').replace('\t',' ').replace('  ',' ').replace('   ',' ').replace('    ',' ')
            texts.append(data)
            file.close() # file is being closed
            
#setting the category of the news as the folder name in which it reside  
        labels.append(os.path.basename(path)) # from the whole path basename only returns the folder name

In [9]:
# creating a pandas dataframe from two lists texts and labels as two columns

df = pd.DataFrame({'category':labels, 'text':texts})

In [10]:
# Printing the data frame for verification purpose

df

,category,text
0,business,Lufthansa flies back to profit German airline ...
1,business,Winn-Dixie files for bankruptcy US supermarket...
2,business,US economy still growing says Fed Most areas o...
3,business,Saab to build Cadillacs in Sweden General Moto...
4,business,Bank voted 8-1 for no rate change The decision...
...,...,...
995,technologie,Mobile games come of age The BBC News website ...
996,technologie,California sets fines for spyware The makers o...
997,technologie,Web helps collect aid donations The web is hel...
998,technologie,Mobiles rack up 20 years of use Mobile phones ...


In [11]:
#take an example from the dataset

df.iloc[5,1]

'Industrial revival hope for Japan Japanese industry is growing faster than expected, boosting hopes that the country\'s retreat back into recession is over. Industrial output rose 2.1% - adjusted for the time of year - in January from a month earlier. At the same time, retail sales picked up faster than at any time since 1997. The news sent Tokyo shares to an eight-month high, as investors hoped for a recovery from the three quarters of contraction seen from April 2004 on. The Nikkei 225 index ended the day up 0.7% at 11,740.60 points, with the yen strengthening 0.7% against the dollar to 104.53 yen. Weaker exports, normally the engine for Japan\'s economy in the face of weak domestic demand, had helped trigger a 0.1% contraction in the final three months of last year after two previous quarters of shrinking GDP. Only an exceptionally strong performance in the early months of 2004 kept the year as a whole from showing a decline. The output figures brought a cautiously optimistic respo

### DOCUMENT CLASSIFICATION

In [12]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords, reuters
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
stop_words = stopwords.words("english")
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import pickle
import nltk
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from warnings import simplefilter
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [13]:
df.sample(10)

,category,text
238,food,3 carrots -- large 1 turnip -- large 2 sta...
165,entertainment,Roxy Music on Isle of Wight bill Roxy Music wi...
971,technologie,'Evil twin' fear for wireless net People using...
836,sport,Gatlin and Hayes win Owen awards American Olym...
231,food,4 cloves garlic finely chopped 1 28 ounce c...
826,sport,"Bekele sets sights on world mark Olympic 10,00..."
989,technologie,Apple laptop is 'greatest gadget' The Apple Po...
259,food,"Firat GÃ¼der admits it, albeit a bit chagrined..."
944,technologie,Domain system scam fear A system to make it ea...
683,politics,'Debate needed' on donations cap A cap on dona...


In [14]:
df['category'].value_counts()

business         100
graphics         100
medical          100
food             100
space            100
entertainment    100
politics         100
sport            100
technologie      100
historical       100
Name: category, dtype: int64

In [15]:
df.iloc[300,1]

"  Tuesday, June 22, 1993 Carderock Division, Naval Surface Warfare Center  (formerly the David Taylor Research Center) Bethesda, Maryland SPONSOR: NESS (Navy Engineering Software System) is sponsoring a one-day Navy Scientific Visualization and Virtual Reality Seminar.  The purpose of the seminar is to present and exchange information for Navy-related scientific visualization and virtual reality programs, research, developments, and applications. PRESENTATIONS: Presentations are solicited on all aspects of Navy-related scientific visualization and virtual reality. All current work, works-in-progress, and proposed work by Navy organizations will be considered. Four types of presentations are available. 1. Regular presentation: 20-30 minutes in length 2. Short presentation: 10 minutes in length 3. Video presentation: a stand-alone videotape (author need not  attend the seminar) 4. Scientific visualization or virtual reality demonstration (BYOH) Accepted presentations will not be publish

In [16]:
#producing train and test sets

x_train,x_test,y_train,y_test = train_test_split(df['text'],df["category"],test_size = 0.2)
train_index = x_train.index
test_index = x_test.index

#### Model 1 - Count Vectorizer

In [17]:
#Vector space modelling
vectorizer = CountVectorizer(analyzer = "word")
count_vectorizer_model = vectorizer.fit(df["text"])
vectors = count_vectorizer_model.transform(df["text"])

#Modelling
x_train = vectors[train_index,:]
y_train = df['category'][train_index]  
x_test  = vectors[test_index,:] 
y_test  = df['category'][test_index]   

clf = RandomForestClassifier()
clf.fit(x_train, y_train)
predictions = clf.predict(x_test)
    
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

predictions_1 = accuracy_score(y_test, predictions)

               precision    recall  f1-score   support

     business       0.62      0.83      0.71        18
entertainment       0.69      0.72      0.71        25
         food       0.75      0.86      0.80        14
     graphics       0.60      0.71      0.65        17
   historical       0.85      0.92      0.88        24
      medical       0.68      0.72      0.70        18
     politics       0.88      0.68      0.77        22
        space       1.00      0.75      0.86        20
        sport       0.90      0.90      0.90        20
  technologie       0.82      0.64      0.72        22

     accuracy                           0.77       200
    macro avg       0.78      0.77      0.77       200
 weighted avg       0.79      0.77      0.77       200

[[15  2  0  0  1  0  0  0  0  0]
 [ 4 18  0  0  0  1  0  0  1  1]
 [ 0  0 12  2  0  0  0  0  0  0]
 [ 0  0  2 12  0  2  0  0  0  1]
 [ 0  0  1  0 22  0  1  0  0  0]
 [ 0  1  1  2  1 13  0  0  0  0]
 [ 2  1  0  1  2  0 15  0  1 

C:\Users\fetih.kaya\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


#### Model 2 - Tf-idf Vectorizer

In [18]:
#Vector space modelling
vectorizer = TfidfVectorizer(analyzer = "word")
tfidf_model = vectorizer.fit(df["text"])
vectors = tfidf_model.transform(df["text"])

#Modelling
x_train = vectors[train_index,:]
y_train = df['category'][train_index]  
x_test  = vectors[test_index,:] 
y_test  = df['category'][test_index]   

clf = RandomForestClassifier()
clf.fit(x_train, y_train)
predictions = clf.predict(x_test)
    
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

predictions_2 = accuracy_score(y_test, predictions)

               precision    recall  f1-score   support

     business       0.50      0.72      0.59        18
entertainment       0.81      0.84      0.82        25
         food       0.89      0.57      0.70        14
     graphics       0.34      0.71      0.46        17
   historical       0.80      0.67      0.73        24
      medical       0.73      0.61      0.67        18
     politics       0.75      0.68      0.71        22
        space       0.78      0.70      0.74        20
        sport       0.95      0.95      0.95        20
  technologie       0.64      0.32      0.42        22

     accuracy                           0.68       200
    macro avg       0.72      0.68      0.68       200
 weighted avg       0.72      0.68      0.69       200

[[13  1  0  1  0  0  1  0  0  2]
 [ 2 21  0  1  0  0  1  0  0  0]
 [ 1  0  8  4  1  0  0  0  0  0]
 [ 1  0  0 12  0  0  0  4  0  0]
 [ 1  1  1  3 16  0  1  0  0  1]
 [ 0  0  0  7  0 11  0  0  0  0]
 [ 1  2  0  0  2  1 15  0  0 

C:\Users\fetih.kaya\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


##### Model 3 - Count Vectorizer + StopWords

In [19]:
#Vector space modelling
vectorizer = CountVectorizer(analyzer = "word", stop_words='english')
count_vectorizer_model = vectorizer.fit(df["text"])
vectors = count_vectorizer_model.transform(df["text"])

#Modelling
x_train = vectors[train_index,:]
y_train = df['category'][train_index]  
x_test  = vectors[test_index,:] 
y_test  = df['category'][test_index]   

clf = RandomForestClassifier()
clf.fit(x_train, y_train)
predictions = clf.predict(x_test)
    
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

predictions_3 = accuracy_score(y_test, predictions)

               precision    recall  f1-score   support

     business       0.64      1.00      0.78        18
entertainment       0.86      0.96      0.91        25
         food       1.00      0.79      0.88        14
     graphics       0.57      0.71      0.63        17
   historical       1.00      0.83      0.91        24
      medical       0.62      0.72      0.67        18
     politics       0.90      0.82      0.86        22
        space       0.76      0.80      0.78        20
        sport       0.91      1.00      0.95        20
  technologie       0.88      0.32      0.47        22

     accuracy                           0.80       200
    macro avg       0.81      0.79      0.78       200
 weighted avg       0.82      0.80      0.79       200

[[18  0  0  0  0  0  0  0  0  0]
 [ 0 24  0  0  0  0  0  0  1  0]
 [ 0  0 11  2  0  0  0  1  0  0]
 [ 1  0  0 12  0  4  0  0  0  0]
 [ 0  1  0  0 20  0  0  2  0  1]
 [ 0  0  0  2  0 13  1  2  0  0]
 [ 3  0  0  1  0  0 18  0  0 

C:\Users\fetih.kaya\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


##### Model 4 - Tf - idf Vectorizer + StopWords

In [20]:
#Vector space modelling
vectorizer = TfidfVectorizer(analyzer = "word", stop_words='english')
tfidf_model = vectorizer.fit(df["text"])
vectors = tfidf_model.transform(df["text"])

#Modelling
x_train = vectors[train_index,:]
y_train = df['category'][train_index]  
x_test  = vectors[test_index,:] 
y_test  = df['category'][test_index]   

clf = RandomForestClassifier()
clf.fit(x_train, y_train)
predictions = clf.predict(x_test)
    
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

predictions_4= accuracy_score(y_test, predictions)

               precision    recall  f1-score   support

     business       0.68      0.94      0.79        18
entertainment       0.95      0.80      0.87        25
         food       1.00      0.79      0.88        14
     graphics       0.48      0.82      0.61        17
   historical       0.91      0.88      0.89        24
      medical       0.71      0.83      0.77        18
     politics       0.95      0.86      0.90        22
        space       0.89      0.80      0.84        20
        sport       0.91      1.00      0.95        20
  technologie       1.00      0.45      0.62        22

     accuracy                           0.81       200
    macro avg       0.85      0.82      0.81       200
 weighted avg       0.86      0.81      0.82       200

[[17  1  0  0  0  0  0  0  0  0]
 [ 2 20  0  1  1  0  0  0  1  0]
 [ 0  0 11  1  0  1  1  0  0  0]
 [ 0  0  0 14  0  3  0  0  0  0]
 [ 0  0  0  2 21  1  0  0  0  0]
 [ 0  0  0  2  0 15  0  1  0  0]
 [ 0  0  0  1  1  0 19  1  0 

C:\Users\fetih.kaya\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


##### Model 5 - Count Vectorizer + StopWords + Stemming

In [21]:
#Stemming
df_new = df
porter = nltk.PorterStemmer()

for i in range(1000):
    df_new['text'][i] = df['text'][i].lower().split(' ')
    df_new['text'][i] = [porter.stem(t) for t in df_new['text'][i]]
    df_new['text'][i] = ' '.join(df_new['text'][i])
    
#Vector space modelling
vectorizer = CountVectorizer(analyzer = "word", stop_words='english')
count_vectorizer_model = vectorizer.fit(df_new["text"])
vectors = count_vectorizer_model.transform(df_new["text"])

#Modelling
x_train = vectors[train_index,:]
y_train = df['category'][train_index]  
x_test  = vectors[test_index,:] 
y_test  = df['category'][test_index]   

clf = RandomForestClassifier()
clf.fit(x_train, y_train)
predictions = clf.predict(x_test)
    
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

predictions_5 = accuracy_score(y_test, predictions)

               precision    recall  f1-score   support

     business       0.68      0.94      0.79        18
entertainment       0.95      0.80      0.87        25
         food       1.00      1.00      1.00        14
     graphics       0.64      0.82      0.72        17
   historical       0.95      0.88      0.91        24
      medical       0.83      0.83      0.83        18
     politics       0.95      0.91      0.93        22
        space       0.76      0.80      0.78        20
        sport       0.95      1.00      0.98        20
  technologie       0.80      0.55      0.65        22

     accuracy                           0.84       200
    macro avg       0.85      0.85      0.85       200
 weighted avg       0.86      0.84      0.84       200

[[17  0  0  0  0  0  0  0  0  1]
 [ 2 20  0  0  0  0  1  1  1  0]
 [ 0  0 14  0  0  0  0  0  0  0]
 [ 0  0  0 14  0  2  0  1  0  0]
 [ 0  0  0  1 21  0  0  1  0  1]
 [ 0  0  0  1  0 15  0  2  0  0]
 [ 0  0  0  1  0  0 20  0  0 

C:\Users\fetih.kaya\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


##### Model 6 - Tf - idf Vectorizer + StopWords + Stemming

In [22]:
#Stemming
df_new = df
porter = nltk.PorterStemmer()

for i in range(1000):
    df_new['text'][i] = df['text'][i].lower().split(' ')
    df_new['text'][i] = [porter.stem(t) for t in df_new['text'][i]]
    df_new['text'][i] = ' '.join(df_new['text'][i])

#Vector space modelling
vectorizer = TfidfVectorizer(analyzer = "word", stop_words='english')
tfidf_model = vectorizer.fit(df_new["text"])
vectors = tfidf_model.transform(df_new["text"])

#Modelling
x_train = vectors[train_index,:]
y_train = df['category'][train_index]  
x_test  = vectors[test_index,:] 
y_test  = df['category'][test_index]   

clf = RandomForestClassifier()
clf.fit(x_train, y_train)
predictions = clf.predict(x_test)
    
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

predictions_6 = accuracy_score(y_test, predictions)

               precision    recall  f1-score   support

     business       0.50      0.72      0.59        18
entertainment       0.83      0.80      0.82        25
         food       0.67      0.86      0.75        14
     graphics       0.71      0.88      0.79        17
   historical       1.00      0.79      0.88        24
      medical       1.00      0.78      0.88        18
     politics       0.90      0.86      0.88        22
        space       0.80      0.80      0.80        20
        sport       0.90      0.95      0.93        20
  technologie       0.88      0.64      0.74        22

     accuracy                           0.81       200
    macro avg       0.82      0.81      0.81       200
 weighted avg       0.83      0.81      0.81       200

[[13  2  0  0  0  0  2  0  0  1]
 [ 2 20  0  1  0  0  0  0  2  0]
 [ 1  0 12  1  0  0  0  0  0  0]
 [ 0  0  2 15  0  0  0  0  0  0]
 [ 1  2  0  1 19  0  0  1  0  0]
 [ 0  0  1  0  0 14  0  3  0  0]
 [ 2  0  0  1  0  0 19  0  0 

C:\Users\fetih.kaya\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


### Determination for parameters for Count Vectorizer, Tf-idf Vectorizer and Random Forest

In [23]:
#Random forest parameter selection

def report(results, n_top=30): #print all results
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

#Random Forest random search
clf = RandomForestClassifier()

param_grid = {"criterion": ["gini", "entropy"],
              "n_estimators": [5, 10, 20, 50, 100, 200, 400],
              "min_samples_split": [2, 5, 10, 20, 50],
              "max_depth": [None, 2, 5, 10, 20, 50, 100],
              "max_features": [None, "auto", "sqrt", "log2"]
             }

n_iter_search = 20

random_search = RandomizedSearchCV(clf, param_distributions=param_grid, n_iter=n_iter_search, cv=5)

simplefilter('ignore')

random_search.fit(x_train, y_train)

report(random_search.cv_results_)

Model with rank: 1
Mean validation score: 0.940 (std: 0.011)
Parameters: {'n_estimators': 400, 'min_samples_split': 10, 'max_features': 'sqrt', 'max_depth': 50, 'criterion': 'gini'}

Model with rank: 2
Mean validation score: 0.939 (std: 0.015)
Parameters: {'n_estimators': 200, 'min_samples_split': 2, 'max_features': 'log2', 'max_depth': 100, 'criterion': 'gini'}

Model with rank: 2
Mean validation score: 0.939 (std: 0.010)
Parameters: {'n_estimators': 400, 'min_samples_split': 50, 'max_features': 'auto', 'max_depth': 50, 'criterion': 'gini'}

Model with rank: 4
Mean validation score: 0.932 (std: 0.017)
Parameters: {'n_estimators': 200, 'min_samples_split': 20, 'max_features': 'sqrt', 'max_depth': 20, 'criterion': 'gini'}

Model with rank: 5
Mean validation score: 0.919 (std: 0.008)
Parameters: {'n_estimators': 200, 'min_samples_split': 10, 'max_features': 'log2', 'max_depth': 20, 'criterion': 'gini'}

Model with rank: 6
Mean validation score: 0.902 (std: 0.021)
Parameters: {'n_estimato

In [25]:
#Count Vectorizer parameter selection

simplefilter(action='ignore', category=FutureWarning)

y = 0

dataframe_1 = pd.DataFrame(np.zeros((900, 5)), columns=['decode_error', 'strip_accents',
                                                      'max_df', 'max_features', 'accuracy'])

decode_error = ("strict", "ignore", "replace")
strip_accents = ("ascii", "unicode",None)
max_df = ( 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0)
max_features = ( 10, 20, 50, 100, 500, 1000, 5000, 10000, 20000, None)

for i in decode_error:
    for j in strip_accents:
        for l in max_df:
            for m in max_features:
                #Count vectorizer modelling
                vectorizer = CountVectorizer(analyzer = "word", decode_error="i", strip_accents=j, 
                                                 max_df=l, max_features=m)
                count_vectorizer_model = vectorizer.fit(df["text"])
                vectors = count_vectorizer_model.transform(df["text"])

                #Modelling
                x_train = vectors[train_index,:]
                y_train = df['category'][train_index]  
                x_test  = vectors[test_index,:] 
                y_test  = df['category'][test_index]   

                clf = RandomForestClassifier(criterion='gini', n_estimators=400, min_samples_split=10, 
                                             max_depth=50, max_features='sqrt')
                clf.fit(x_train, y_train)
                predictions = clf.predict(x_test)
                    
                dataframe_1.iloc[y,0] = i
                dataframe_1.iloc[y,1] = j
                dataframe_1.iloc[y,2] = l
                dataframe_1.iloc[y,3] = m
                dataframe_1.iloc[y,4] = accuracy_score(y_test, predictions)
                    
                y = y+1
                    
dataframe_1  = dataframe_1.sort_values(["accuracy"], ascending = False)  

dataframe_1.head(10)

,decode_error,strip_accents,max_df,max_features,accuracy
709,replace,unicode,0.1,NaN,0.960
108,strict,unicode,0.1,20000.0,0.955
809,replace,None,0.1,NaN,0.955
78,strict,ascii,0.8,20000.0,0.950
409,ignore,unicode,0.1,NaN,0.950
109,strict,unicode,0.1,NaN,0.950
137,strict,unicode,0.4,10000.0,0.950
509,ignore,None,0.1,NaN,0.945
708,replace,unicode,0.1,20000.0,0.945
119,strict,unicode,0.2,NaN,0.945


In [26]:
#Tf-idf Vectorizer parameter selection

simplefilter(action='ignore', category=FutureWarning)

y = 0

dataframe_2 = pd.DataFrame(np.zeros((900, 5)), columns=['decode_error', 'strip_accents',
                                                      'max_df', 'max_feature', 'accuracy'])

decode_error = ("strict", "ignore", "replace")
strip_accents = ("ascii", "unicode",None)
max_df = ( 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0)
max_features = ( 10, 20, 50, 100, 500, 1000, 5000, 10000, 20000, None)

for i in decode_error:
    for j in strip_accents:
        for l in max_df:
            for m in max_features:
                #Tf - idf vectorizer modelling
                vectorizer = TfidfVectorizer(analyzer = "word", decode_error="i", strip_accents=j, 
                                                 max_df=l, max_features=m)
                tfidf_model = vectorizer.fit(df["text"])
                vectors = tfidf_model.transform(df["text"])

                #Modelling
                x_train = vectors[train_index,:]
                y_train = df['category'][train_index]  
                x_test  = vectors[test_index,:] 
                y_test  = df['category'][test_index]   

                clf = RandomForestClassifier(criterion='gini', n_estimators=400, min_samples_split=10, 
                                             max_depth=50, max_features='sqrt')
                clf.fit(x_train, y_train)
                predictions = clf.predict(x_test)
                    
                dataframe_2.iloc[y,0] = i
                dataframe_2.iloc[y,1] = j
                dataframe_2.iloc[y,2] = l
                dataframe_2.iloc[y,3] = m
                dataframe_2.iloc[y,4] = accuracy_score(y_test, predictions)
                    
                y = y+1
                    
dataframe_2  = dataframe_2.sort_values(["accuracy"], ascending = False)   

dataframe_2.head(10)

,decode_error,strip_accents,max_df,max_feature,accuracy
8,strict,ascii,0.1,20000.0,0.970
109,strict,unicode,0.1,NaN,0.965
718,replace,unicode,0.2,20000.0,0.965
608,replace,ascii,0.1,20000.0,0.965
38,strict,ascii,0.4,20000.0,0.965
209,strict,None,0.1,NaN,0.965
107,strict,unicode,0.1,10000.0,0.960
219,strict,None,0.2,NaN,0.960
438,ignore,unicode,0.4,20000.0,0.960
319,ignore,ascii,0.2,NaN,0.960


##### Model 7 - Count Vectorizer + StopWords + Stemming + Random Forest Hyper-parameter Tuning

In [27]:
#Stemming
df_new = df
porter = nltk.PorterStemmer()

for i in range(1000):
    df_new['text'][i] = df['text'][i].lower().split(' ')
    df_new['text'][i] = [porter.stem(t) for t in df_new['text'][i]]
    df_new['text'][i] = ' '.join(df_new['text'][i])
    
#Vector space modelling
vectorizer = CountVectorizer(analyzer = "word", stop_words='english')
count_vectorizer_model = vectorizer.fit(df_new["text"])
vectors = count_vectorizer_model.transform(df_new["text"])

#Modelling
x_train = vectors[train_index,:]
y_train = df['category'][train_index]  
x_test  = vectors[test_index,:] 
y_test  = df['category'][test_index]   

clf = RandomForestClassifier(criterion='gini', n_estimators=400, min_samples_split=10, max_depth=50, max_features='sqrt')
clf.fit(x_train, y_train)
predictions = clf.predict(x_test)
    
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

predictions_7 = accuracy_score(y_test, predictions)

               precision    recall  f1-score   support

     business       0.90      1.00      0.95        18
entertainment       0.96      0.96      0.96        25
         food       1.00      1.00      1.00        14
     graphics       0.74      0.82      0.78        17
   historical       1.00      0.96      0.98        24
      medical       0.81      0.94      0.87        18
     politics       0.92      1.00      0.96        22
        space       1.00      0.85      0.92        20
        sport       0.95      0.95      0.95        20
  technologie       1.00      0.77      0.87        22

     accuracy                           0.93       200
    macro avg       0.93      0.93      0.92       200
 weighted avg       0.93      0.93      0.93       200

[[18  0  0  0  0  0  0  0  0  0]
 [ 0 24  0  0  0  0  0  0  1  0]
 [ 0  0 14  0  0  0  0  0  0  0]
 [ 0  0  0 14  0  3  0  0  0  0]
 [ 0  0  0  0 23  1  0  0  0  0]
 [ 0  0  0  1  0 17  0  0  0  0]
 [ 0  0  0  0  0  0 22  0  0 

##### Model 8 - Tf - idf Vectorizer + StopWords + Stemming + Random Forest Hyper-parameter Tuning

In [28]:
#Stemming
df_new = df
porter = nltk.PorterStemmer()

for i in range(1000):
    df_new['text'][i] = df['text'][i].lower().split(' ')
    df_new['text'][i] = [porter.stem(t) for t in df_new['text'][i]]
    df_new['text'][i] = ' '.join(df_new['text'][i])

#Vector space modelling
vectorizer = TfidfVectorizer(analyzer = "word", stop_words='english')
tfidf_model = vectorizer.fit(df_new["text"])
vectors = tfidf_model.transform(df_new["text"])

#Modelling
x_train = vectors[train_index,:]
y_train = df['category'][train_index]  
x_test  = vectors[test_index,:] 
y_test  = df['category'][test_index]   

clf = RandomForestClassifier(criterion='gini', n_estimators=400, min_samples_split=10, max_depth=50, max_features='sqrt')
clf.fit(x_train, y_train)
predictions = clf.predict(x_test)
    
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

predictions_8 = accuracy_score(y_test, predictions)

               precision    recall  f1-score   support

     business       0.90      1.00      0.95        18
entertainment       0.93      1.00      0.96        25
         food       1.00      1.00      1.00        14
     graphics       0.88      0.88      0.88        17
   historical       1.00      0.96      0.98        24
      medical       0.90      1.00      0.95        18
     politics       0.96      1.00      0.98        22
        space       0.95      0.90      0.92        20
        sport       1.00      0.95      0.97        20
  technologie       1.00      0.82      0.90        22

     accuracy                           0.95       200
    macro avg       0.95      0.95      0.95       200
 weighted avg       0.95      0.95      0.95       200

[[18  0  0  0  0  0  0  0  0  0]
 [ 0 25  0  0  0  0  0  0  0  0]
 [ 0  0 14  0  0  0  0  0  0  0]
 [ 0  0  0 15  0  1  0  1  0  0]
 [ 0  1  0  0 23  0  0  0  0  0]
 [ 0  0  0  0  0 18  0  0  0  0]
 [ 0  0  0  0  0  0 22  0  0 

##### Model 9 - Count Vectorizer + StopWords + Stemming + Random Forest Hyper-parameter Tuning + Count Vectorizer Hyper-parameter Tuning

In [30]:
#Stemming
df_new = df
porter = nltk.PorterStemmer()

for i in range(1000):
    df_new['text'][i] = df['text'][i].lower().split(' ')
    df_new['text'][i] = [porter.stem(t) for t in df_new['text'][i]]
    df_new['text'][i] = ' '.join(df_new['text'][i])
    
#Vector space modelling
vectorizer = CountVectorizer(analyzer = "word", stop_words='english', decode_error='replace', strip_accents='unicode', 
                             max_df=0.1, max_features=None)
count_vectorizer_model = vectorizer.fit(df_new["text"])
vectors = count_vectorizer_model.transform(df_new["text"])

#Modelling
x_train = vectors[train_index,:]
y_train = df['category'][train_index]  
x_test  = vectors[test_index,:] 
y_test  = df['category'][test_index]   

clf = RandomForestClassifier(criterion='gini', n_estimators=400, min_samples_split=10, max_depth=50, max_features='sqrt')
clf.fit(x_train, y_train)
predictions = clf.predict(x_test)
    
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

predictions_9 = accuracy_score(y_test, predictions)

               precision    recall  f1-score   support

     business       0.86      1.00      0.92        18
entertainment       1.00      1.00      1.00        25
         food       1.00      1.00      1.00        14
     graphics       0.74      1.00      0.85        17
   historical       1.00      0.96      0.98        24
      medical       1.00      1.00      1.00        18
     politics       1.00      0.95      0.98        22
        space       1.00      0.90      0.95        20
        sport       1.00      0.95      0.97        20
  technologie       1.00      0.82      0.90        22

     accuracy                           0.95       200
    macro avg       0.96      0.96      0.96       200
 weighted avg       0.96      0.95      0.96       200

[[18  0  0  0  0  0  0  0  0  0]
 [ 0 25  0  0  0  0  0  0  0  0]
 [ 0  0 14  0  0  0  0  0  0  0]
 [ 0  0  0 17  0  0  0  0  0  0]
 [ 0  0  0  1 23  0  0  0  0  0]
 [ 0  0  0  0  0 18  0  0  0  0]
 [ 0  0  0  1  0  0 21  0  0 

##### Model 10 - Tf - idf Vectorizer + StopWords + Stemming + Random Forest Hyper-parameter Tuning + Tf-idf Vectorizer Hyper-parameter Tuning

In [31]:
#Stemming
df_new = df
porter = nltk.PorterStemmer()

for i in range(1000):
    df_new['text'][i] = df['text'][i].lower().split(' ')
    df_new['text'][i] = [porter.stem(t) for t in df_new['text'][i]]
    df_new['text'][i] = ' '.join(df_new['text'][i])

#Vector space modelling
vectorizer = TfidfVectorizer(analyzer = "word", stop_words='english', decode_error='strict', strip_accents='ascii', 
                             max_df=0.1, max_features=20000)
tfidf_model = vectorizer.fit(df_new["text"])
vectors = tfidf_model.transform(df_new["text"])

#Modelling
x_train = vectors[train_index,:]
y_train = df['category'][train_index]  
x_test  = vectors[test_index,:] 
y_test  = df['category'][test_index]   

clf = RandomForestClassifier(criterion='gini', n_estimators=400, min_samples_split=10, max_depth=50, max_features='sqrt')
clf.fit(x_train, y_train)
predictions = clf.predict(x_test)
    
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

predictions_10 = accuracy_score(y_test, predictions)

               precision    recall  f1-score   support

     business       1.00      1.00      1.00        18
entertainment       0.96      1.00      0.98        25
         food       1.00      1.00      1.00        14
     graphics       0.84      0.94      0.89        17
   historical       1.00      0.96      0.98        24
      medical       1.00      1.00      1.00        18
     politics       1.00      1.00      1.00        22
        space       1.00      0.90      0.95        20
        sport       0.95      0.95      0.95        20
  technologie       0.95      0.95      0.95        22

     accuracy                           0.97       200
    macro avg       0.97      0.97      0.97       200
 weighted avg       0.97      0.97      0.97       200

[[18  0  0  0  0  0  0  0  0  0]
 [ 0 25  0  0  0  0  0  0  0  0]
 [ 0  0 14  0  0  0  0  0  0  0]
 [ 0  0  0 16  0  0  0  0  0  1]
 [ 0  0  0  0 23  0  0  0  1  0]
 [ 0  0  0  0  0 18  0  0  0  0]
 [ 0  0  0  0  0  0 22  0  0 

### Comparison

In [36]:
Models = pd.DataFrame(["Model 1", "Model 2", "Model 3", "Model 4", "Model 5", 
          "Model 6", "Model 7", "Model 8", "Model 9", "Model 10"], columns=['Models'])

Conducted_Steps = pd.DataFrame(["Count vectorizer", "Tf-idf Vectorizer", "Count vectorizer + Stopwords removal", 
                   "Tf-idf Vectorizer + Stopwords removal", "Count vectorizer + Stopwords removal + Stemming", 
                   "Tf-idf Vectorizer + Stopwords removal + Stemming", 
                   "Count vectorizer + Stopwords removal + Stemming + Random Forest Parameter Tuning",
                   "Tf-idf Vectorizer + Stopwords removal + Stemming + Random Forest Parameter Tuning",
                   "Count vectorizer + Stopwords removal + Stemming + Random Forest Parameter Tuning + Count Vectorizer Parameter Tuning",
                   "Tf-idf Vectorizer + Stopwords removal + Stemming + Random Forest Parameter Tuning + Tf-idf Vectorizer Parameter Tuning"], columns=['Conducted Steps'])

Accuracies = pd.DataFrame([predictions_1, predictions_2, predictions_3, predictions_4, predictions_5,
             predictions_6, predictions_7, predictions_8, predictions_9, predictions_10], columns=['Accuracies'])

comparison  = pd.concat([Models, Conducted_Steps, Accuracies], axis=1)

pd.set_option('max_colwidth', 400)


comparison

,Models,Conducted Steps,Accuracies
0,Model 1,Count vectorizer,0.770
1,Model 2,Tf-idf Vectorizer,0.680
2,Model 3,Count vectorizer + Stopwords removal,0.795
3,Model 4,Tf-idf Vectorizer + Stopwords removal,0.815
4,Model 5,Count vectorizer + Stopwords removal + Stemming,0.845
5,Model 6,Tf-idf Vectorizer + Stopwords removal + Stemming,0.805
6,Model 7,Count vectorizer + Stopwords removal + Stemming + Random Forest Parameter Tuning,0.925
7,Model 8,Tf-idf Vectorizer + Stopwords removal + Stemming + Random Forest Parameter Tuning,0.950
8,Model 9,Count vectorizer + Stopwords removal + Stemming + Random Forest Parameter Tuning + Count Vectorizer Parameter Tuning,0.955
9,Model 10,Tf-idf Vectorizer + Stopwords removal + Stemming + Random Forest Parameter Tuning + Tf-idf Vectorizer Parameter Tuning,0.970
